In [61]:
import numpy as np
import sys
from scipy.special import expit as sigmoid

training_data_path = sys.argv[1]
testing_data_path = sys.argv[2]
output_path = sys.argv[3]

# training_data_path = "../data/devnagri_train.csv"
# testing_data_path = "../data/devnagri_test_public.csv"
# output_path = "../data/nn/b/cs1160328"

batch_size = 128
n0 = 5
activation = 'sigmoid'
hidden_layers_sizes = [100,50]

IndexError: list index out of range

In [2]:
def relu(x):
    return (x>0) * x

def tanh(x):
    return np.tanh(x)

def reluPrime(x):
    return (x>0)+0

def tanhPrime(x):
    return 1 - np.power(x,2)

def sigmoidPrime(x):
    return x * (1 - x)

def exp_normalize(x):
    b = np.amax(x,axis=1,keepdims = True)
    y = np.exp(x - b)
    return y / y.sum(axis=1,keepdims=True)

In [57]:
class NeuralNetwork:
    
    def __init__(self,input_size,output_size,hidden_layers_sizes, activation):
        self.weights = []
        self.biases = []
        
        if(activation == 'relu'):
            self.activation = relu
            self.activationPrime = reluPrime
        elif(activation == 'tanh'):
            self.activation = tanh
            self.activationPrime = tanhPrime
        else:
            self.activation = sigmoid
            self.activationPrime = sigmoidPrime
        
        self.input_size = input_size
        self.output_size = output_size
        self.hiddent_layers_sizes = hidden_layers_sizes
        
        prev_layer_count = input_size
        
        for i in range(len(hidden_layers_sizes) + 1):
            if i==len(hidden_layers_sizes):
                self.weights.append(np.random.rand(prev_layer_count, output_size)/100)
                self.biases.append(np.random.rand(1, output_size)/100)        
            else:
                hidden_layer_count = hidden_layers_sizes[i]
                self.weights.append(np.random.rand(prev_layer_count, hidden_layer_count)/100)
                self.biases.append(np.random.rand(1, hidden_layer_count)/100)
                prev_layer_count = hidden_layer_count
        
    def train(self,inpX,inpY,batch_size,n0,max_iterations):
        max_examples = inpX.shape[0]
        max_possible_iterations = int(0.5 + max_examples / batch_size)
        num_hidden_layers = len(self.weights) - 1
               
        count = 0
            
        lr = n0
        totLoss = 0
        prevAvgLoss = sys.float_info.max
        epoch = 0
        
        for n in range(max_iterations):
            # Forming Mini Batches
            i_eff = n%max_possible_iterations
            
            # Updating Learning Rate
            if (i_eff == 0 and n!=0):
                avgLoss = totLoss/max_possible_iterations
                if(avgLoss >= prevAvgLoss):
                    count += 1
                    lr = n0 / np.sqrt(count+1)
#                 print("Epoch = ",epoch," Average Loss = ",avgLoss," New Learning Rate = ",lr)
                epoch += 1
                prevAvgLoss = avgLoss
                totLoss = 0
            
            outputs = []
            
            if i_eff != max_possible_iterations - 1:
                X = inpX[i_eff*batch_size: (i_eff+1)*batch_size]
                Y = inpY[i_eff*batch_size: (i_eff+1)*batch_size]
            else:
                X = inpX[i_eff*batch_size:]
                Y = inpY[i_eff*batch_size:]
                
#             # Neural Network Forward Propagation (Cross Entropy)
#             outputs.append(X)
#             prev_layer_output = X
#             for i in range(num_hidden_layers + 1):
#                 weight = self.weights[i]
#                 bias = self.biases[i]
#                 if i == num_hidden_layers:
#                     prev_layer_output = exp_normalize(prev_layer_output.dot(weight) + bias)
#                 else:
#                     prev_layer_output = self.activation(prev_layer_output.dot(weight) + bias)
                    
#                 outputs.append(prev_layer_output)
            
#             # Backpropagation
#             dWs = []
#             dbs = []
            
#             for i in range(num_hidden_layers + 1,0,-1):
#                 if i == num_hidden_layers + 1:
#                     delta = outputs[i].copy()
#                     delta[range(Y.shape[0]),Y] -= 1
#                 else:
#                     delta = delta.dot(self.weights[i].T) * self.activationPrime(outputs[i])
#                 dW = (outputs[i-1].T).dot(delta)
#                 dWs.append(dW)
#                 dbs.append(np.sum(delta,axis=0,keepdims=True))
#             if (n%100 == 0):
#                 loss_ = np.sum(-1*np.log(outputs[-1][range(Y.shape[0]),Y] + 0.001)) / Y.shape[0]
#                 labels_ = np.argmax(outputs[-1],axis = 1)
#                 accuracy_ = 100 * np.sum(labels_ == Y)/Y.shape[0]
#                 print("Iteration ",n,"\tLoss = ",loss_,"\tAccuracy = ",accuracy_,"%")
#             dWs.reverse()
#             dbs.reverse()

#             # Gradient Descent Parameter Update
#             for i in range(len(dWs)):
#                 self.weights[i] += dWs[i].dot(-1 * lr)
#                 self.biases[i] += dbs[i].dot(-1 * lr)
            
#             loss = np.sum(-1*np.log(outputs[-1][range(Y.shape[0]),Y] + 0.001)) / Y.shape[0]
#             totLoss += loss

             # Neural Network Forward Propagation (MSE)
            outputs.append(X)
            prev_layer_output = X
            for i in range(num_hidden_layers + 1):
                weight = self.weights[i]
                bias = self.biases[i]
                if i == num_hidden_layers:
                    prev_layer_output = sigmoid(prev_layer_output.dot(weight) + bias)
                else:
                    prev_layer_output = self.activation(prev_layer_output.dot(weight) + bias)
                outputs.append(prev_layer_output)
            
            # Backpropagation
            dWs = []
            dbs = []
            
            y_onehot = np.zeros((Y.shape[0],self.output_size))
            y_onehot[range(Y.shape[0]),Y] = 1
            
            for i in range(num_hidden_layers + 1,0,-1):
                if i == num_hidden_layers + 1:
                    delta = (outputs[i] - y_onehot).dot(2/Y.shape[0]) * sigmoidPrime(outputs[i])
                else:
                    delta = delta.dot(self.weights[i].T) * self.activationPrime(outputs[i])
                dW = (outputs[i-1].T).dot(delta)
                dWs.append(dW)
                dbs.append(np.sum(delta,axis=0,keepdims=True))

#             if (n%100 == 0):
#                 loss = np.sum(np.power(outputs[-1] - y_onehot,2) )/Y.shape[0]
#                 labels = np.argmax(outputs[-1],axis = 1)
#                 accuracy = 100 * np.sum(labels == Y)/Y.shape[0]
#                 print("Iteration ",n,"\tLoss = ",loss,"\tAccuracy = ",accuracy,"%")
                
            dWs.reverse()
            dbs.reverse()

            # Gradient Descent Parameter Update
            for i in range(len(dWs)):
                self.weights[i] += dWs[i].dot(-1 * lr)
                self.biases[i] += dbs[i].dot(-1 * lr)

            loss = np.sum(np.power(outputs[-1] - y_onehot,2) )/Y.shape[0]
            totLoss += loss
            
    def predict(self,X):
        return self.forward_run(X)
        
    def forward_run(self,X):
        prev_layer_output = X
        num_hidden_layers = len(self.weights) - 1
        for i in range(num_hidden_layers + 1):
            weight = self.weights[i]
            bias = self.biases[i]
            if i == num_hidden_layers:
                probabilities = exp_normalize(prev_layer_output.dot(weight) + bias)
                labels = np.argmax(probabilities,axis = 1)
                return labels
            else:
                prev_layer_output = self.activation(prev_layer_output.dot(weight) + bias)

In [4]:
def load_data(path,avg,std):
    if avg is None:
        input_data = np.loadtxt(open(path, "rb"), delimiter=",")
        Y = input_data[:,0].copy()
        X = input_data[:,1:].copy()
        avg = np.average(X,axis=0)
        X = X - avg
        std = np.std(X,axis=0)
        std[(std == 0)] = 1
        X = X / std
        return X,Y,avg,std
    else:
        input_data = np.loadtxt(open(path, "rb"), delimiter=",")
        X = input_data[:,1:].copy()
        X = (X - avg)/std
        return X

In [5]:
inpX,Y,avg,std = load_data(training_data_path,None,None)

In [60]:
X = inpX.copy()

input_size = X.shape[1]
output_size = int(np.amax(Y))+1
num_examples = X.shape[0]
max_iterations = int(40*(num_examples/batch_size))

network = NeuralNetwork(input_size,output_size,hidden_layers_sizes,activation)
network.train(X,Y.astype(int),batch_size,n0,max_iterations)

Iteration  0 	Loss =  3.782283867361179 	Accuracy =  2.34375 %
Iteration  100 	Loss =  6.745831338841937 	Accuracy =  2.34375 %
Iteration  200 	Loss =  6.600449216009279 	Accuracy =  0.78125 %
Iteration  300 	Loss =  6.711249824247213 	Accuracy =  2.34375 %
Iteration  400 	Loss =  6.848222235081371 	Accuracy =  0.78125 %
Iteration  500 	Loss =  6.745831338841937 	Accuracy =  2.34375 %
Iteration  600 	Loss =  6.691832870320664 	Accuracy =  3.125 %
Epoch =  0  Average Loss =  6.717183552379846  New Learning Rate =  5
Iteration  700 	Loss =  6.574674708045514 	Accuracy =  2.34375 %
Iteration  800 	Loss =  6.7310345928550674 	Accuracy =  2.34375 %
Iteration  900 	Loss =  6.807452713611032 	Accuracy =  0.78125 %
Iteration  1000 	Loss =  6.799805985538095 	Accuracy =  1.5625 %
Iteration  1100 	Loss =  6.799805973152872 	Accuracy =  1.5625 %
Iteration  1200 	Loss =  6.853780632268569 	Accuracy =  0.78125 %
Epoch =  1  Average Loss =  6.726600477952544  New Learning Rate =  3.5355339059327373


Iteration  10500 	Loss =  6.5336838457943 	Accuracy =  3.125 %
Iteration  10600 	Loss =  6.303698370320086 	Accuracy =  6.25 %
Iteration  10700 	Loss =  6.59125779199172 	Accuracy =  3.90625 %
Iteration  10800 	Loss =  6.598391126089053 	Accuracy =  1.5625 %
Iteration  10900 	Loss =  6.354071421847509 	Accuracy =  1.5625 %
Epoch =  17  Average Loss =  6.585468207314915  New Learning Rate =  1.889822365046136
Iteration  11000 	Loss =  6.690405741679525 	Accuracy =  3.125 %
Iteration  11100 	Loss =  6.522232802547682 	Accuracy =  1.5625 %
Iteration  11200 	Loss =  6.864131440199692 	Accuracy =  0.0 %
Iteration  11300 	Loss =  6.728683618996761 	Accuracy =  1.5625 %
Iteration  11400 	Loss =  6.737519165618606 	Accuracy =  1.5625 %
Iteration  11500 	Loss =  6.695849172365364 	Accuracy =  2.34375 %
Iteration  11600 	Loss =  6.343091710239351 	Accuracy =  3.125 %
Epoch =  18  Average Loss =  6.584652064074546  New Learning Rate =  1.889822365046136
Iteration  11700 	Loss =  6.674040005264967

Epoch =  33  Average Loss =  6.44691413667335  New Learning Rate =  1.4433756729740645
Iteration  20800 	Loss =  6.272380887584987 	Accuracy =  2.34375 %
Iteration  20900 	Loss =  6.597950214612192 	Accuracy =  1.5625 %
Iteration  21000 	Loss =  6.458779471254484 	Accuracy =  3.125 %
Iteration  21100 	Loss =  6.63868699721376 	Accuracy =  1.5625 %
Iteration  21200 	Loss =  6.244568606197561 	Accuracy =  3.125 %
Iteration  21300 	Loss =  6.241833190893663 	Accuracy =  2.34375 %
Epoch =  34  Average Loss =  6.4388410881393  New Learning Rate =  1.4433756729740645
Iteration  21400 	Loss =  6.3766122218349945 	Accuracy =  0.0 %
Iteration  21500 	Loss =  6.654619333329661 	Accuracy =  1.5625 %
Iteration  21600 	Loss =  6.753868159582628 	Accuracy =  0.0 %
Iteration  21700 	Loss =  6.542843893995697 	Accuracy =  0.78125 %
Iteration  21800 	Loss =  5.907336982532781 	Accuracy =  2.34375 %
Iteration  21900 	Loss =  5.974303119315447 	Accuracy =  5.46875 %
Epoch =  35  Average Loss =  6.4431160

In [52]:
# predictions = network.predict(X.copy())
# print(100 * np.sum(predictions == Y)/Y.shape[0])
# print(np.average(predictions))

81.79667519181585


In [53]:
testX = load_data(testing_data_path,avg,std)

In [56]:
predictions = network.predict(testX)
np.savetxt(output_path,predictions,fmt="%i")